# Cook-It-Up-With-the-Fantastic-Four

Python depends on packages for most of its functionality; these can be either built-in (such as sys), or third-party (like all the packages below). Either way you need to import the packages you need before using them.

In [1]:
%matplotlib inline
# See all the "as ..." contructs? They're just aliasing the package names.
# That way we can call methods like plt.plot() instead of matplotlib.pyplot.plot().
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
dftouse=pd.read_csv("submission.csv")
dftouse.head()

,Unnamed: 0,id,ingredients_clean_string,cuisine
0,4987,5,"mushrooms , chopped onion , tomato sauce , che...",mexican
1,9232,7,"minced garlic , brown rice , sour cream , chic...",indian
2,9638,11,"lime juice , sesame oil , garlic cloves , fish...",vietnamese
3,4927,12,"sugar , vanilla extract , corn starch , coffee...",italian
4,3280,13,"frozen pie crust , bourbon whiskey , powdered ...",southern_us


### Create dataframes for Train and Test Data

In [3]:
import json
import pandas as pd

traindf = pd.read_json('train.json')
traindf.head()



,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [17]:
import pandas as pd
from sklearn import tree
from sklearn.feature_extraction import DictVectorizer
from sklearn import preprocessing
from sklearn.linear_model import Ridge


train = pd.DataFrame({'a' : ['a', 'b', 'a'], 'd' : ['e', 'x', 'f'],
                  'b' : [0, 1, 1], 'c' : ['b', 'c', 'b']})

# encode labels
labels = train[['c']]
le = preprocessing.LabelEncoder()
labels_fea = le.fit_transform(labels) 
# vectorize training data
del train['c']
train_as_dicts = [dict(r.iteritems()) for _, r in train.iterrows()]
train_fea = DictVectorizer(sparse=False).fit_transform(train_as_dicts)
# use decision tree
dt = tree.DecisionTreeClassifier()





est = Ridge(alpha=10)
est.fit(train_fea, labels_fea)
predictions = le.inverse_transform(est.predict(train_fea).astype('I'))



#predictions_as_dataframe = train.join(pd.DataFrame({"Prediction": predictions}))
print predictions_as_dataframe

[0 1 0]


/Users/angelhenderson/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/label.py:127: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


IndexError: arrays used as indices must be of integer (or boolean) type

In [16]:
##################################
#  working example
import pandas as pd
from sklearn import tree
from sklearn.feature_extraction import DictVectorizer
from sklearn import preprocessing

train = pd.DataFrame({'a' : ['romaine lettuce', 'black olives', 'a'], 'd' : ['e', 'x', 'f'],
                   'c' : ['b', 'c', 'b']})

labels = train_df[['cuisine']]
le = preprocessing.LabelEncoder()
labels_fea = le.fit_transform(labels) 

# encode labels
labels = train[['c']]
le = preprocessing.LabelEncoder()
labels_fea = le.fit_transform(labels) 
# vectorize training data
del train['c']
train_as_dicts = [dict(r.iteritems()) for _, r in traindf.iterrows()]

sLength = len(traindf['a'])
for a in traindf['ingredients']:
    traindf[a] = Series(np.random.randn(sLength), traindf=df1.index)
train_fea = DictVectorizer(sparse=False).fit_transform(train_as_dicts)
# use decision tree
dt = tree.DecisionTreeClassifier()
dt.fit(train_fea, labels_fea)
# transform result
predictions = le.inverse_transform(dt.predict(train_fea).astype('I'))
predictions_as_dataframe = train.join(pd.DataFrame({"Prediction": predictions}))
print predictions_as_dataframe

NameError: name 'train_df' is not defined

In [ ]:
train = pd.DataFrame({'a' : ['a', 'b', 'a'], 'd' : ['e', 'e', 'f'],
                  'b' : [0, 1, 1], 'c' : ['b', 'c', 'b']})
train.head()

In [ ]:
testdf = pd.read_json('test.json')
testdf.head()